#  [모듈 3.1] HPO 스텝 개발 (Tunning Step)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 1. 모델 튜닝 개요
- 2. 기본 라이브러리 로딩
- 3. 훈련에 사용할 전처리된 파일을 확인
- 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성
- 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행
- 6. HPO 잡 실행 확인 하기
    
---

# 1. 모델 튜닝 개요

하이퍼파라미터 튜닝이라고도 하는 Amazon SageMaker 자동 모델 튜닝은 사용자가 지정한 알고리즘과 다양한 하이퍼파라미터를 사용하여 데이터 세트에 대해 여러 훈련 작업을 실행하여 최적의 모델 버전을 찾습니다. 그런 다음 선택한 지표로 측정된 값에 따라 최적의 성능을 보여준 모델을 만든 하이퍼파라미터 값을 선택합니다.



- 참고
    - 개발자 가이드: [SageMaker 로 자동 모델 튜닝 수행](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/automatic-model-tuning.html)
    - 공식 세이지 메이커의 샘플 입니다. --> [HPO 시작 코드](https://github.com/aws/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/xgboost_direct_marketing/hpo_xgboost_direct_marketing_sagemaker_python_sdk.ipynb)




# 2. 기본 라이브러리 로딩

세이지 메이커 관련 라이브러리를 로딩 합니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

#region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.client("sagemaker")


## 2.1 노트북 변수 로딩


In [2]:
%store -r

# 3. 훈련에 사용할 전처리된 파일을 확인
이후에 훈련에서 사용할 S3의 저장된 전처리 데이터를 확인 합니다.

In [3]:
! aws s3 ls {train_preproc_data_uri} --recursive

2022-03-11 02:36:45     682602 sagemaker-webinar-pipeline-advanced/preporc/train.csv


In [4]:
train_prep_df = pd.read_csv(train_preproc_data_uri)
train_prep_df

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,18052.611626,67152.611626,42,103,1,1,750,3000,93654,...,0,0,1,0,0,0,1,0,1,0
3996,0,34949.202468,51749.202468,23,6,0,3,750,3000,94305,...,0,0,0,1,1,0,0,0,1,0
3997,0,4063.701410,9963.701410,44,35,0,2,750,2550,95476,...,0,0,1,0,0,0,0,1,0,1
3998,0,17390.520451,20490.520451,22,38,0,1,750,3000,90680,...,0,1,0,0,0,0,0,1,0,1


# 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성
- 개발자 가이드의 튜닝 단계 참고 --> [튜닝 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning)



## 4.1 모델 빌딩 파이프라인 변수 생성

파이프라인에서 사용할 파이프라인 파라미터를 정의합니다. 파이프라인을 스케줄하고 실행할 때 파라미터를 이용하여 실행조건을 커스마이징할 수 있습니다. 파라미터를 이용하면 파이프라인 실행시마다 매번 파이프라인 정의를 수정하지 않아도 됩니다.

지원되는 파라미터 타입은 다음과 같습니다:

* `ParameterString` - 파이썬 타입에서 `str` 
* `ParameterInteger` - 파이썬 타입에서 `int` 
* `ParameterFloat` - 파이썬 타입에서 `float` 

이들 파라미터를 정의할 때 디폴트 값을 지정할 수 있으며 파이프라인 실행시 재지정할 수도 있습니다. 지정하는 디폴트 값은 파라미터 타입과 일치하여야 합니다.

본 노트북에서 사용하는 파라미터는 다음과 같습니다.

* `training_instance_type` - 학습작업에서 사용할 `ml.*` 인스턴스 타입
* `training_instance_count` - 학습작업에서 사용할 `ml.*` 인스턴스 갯수
* `input_data` - 입력데이터에 대한 S3 버킷 URI



파이프라인의 각 스텝에서 사용할 변수를 파라미터 변수로서 정의 합니다.


In [5]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

train_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)


train_instance_count = ParameterInteger(
    name="TrainInstanceCount",
    default_value=1
)

input_data = ParameterString(
    name="InputData",
    default_value=train_preproc_data_uri,
)


## 4.2 Estimator 생성

Estimator 생성시에 인자가 필요 합니다. 주요한 인자만 보겠습니다.
- 사용자 훈련 코드 ""xgboost_script.py"
- 훈련이 끝난 후에 결과인 모델 아티펙트의 경로 "estimator_output_path" 지정 합니다. 지정 안할 시에는 디폴트 경로로 저장 됩니다.


In [6]:
from sagemaker.xgboost.estimator import XGBoost

estimator_output_path = f's3://{bucket}/{project_prefix}/tuning_jobs'
print("estimator_output_path: \n", estimator_output_path)

xgb_estimator = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = train_instance_type,
    framework_version = "1.0-1")

estimator_output_path: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-webinar-pipeline-advanced/tuning_jobs


## 4.3 고정(Static) 하이퍼파라미터 세팅

기본 XGBoost 하이퍼파라미터 외에 `scale_pos_weight` 는 레이블이 뷸균형이 되어 있을 경우에, 레이블 값의 가중치를 부여하는 파라미터 입니다. 레이블 0, 1 의 비율에 따라 지정합니다.

In [7]:
from src.p_utils import get_pos_scale_weight
class_weight = get_pos_scale_weight(train_prep_df, label='fraud')

fraud_sum: 131 , non_fraud_sum: 3869, class_weight: 29


In [8]:
hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "objective": "binary:logistic",
        "num_round": "100",
}



## 4.4 튜닝할 하이퍼파라미터 범위 설정
여기서는 `eta, min_child_weight, alpha, max_depth` 를 튜닝 합니다.

In [9]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}



## 4.5 튜너 설정 및 생성
- xbg_estimator 정의된  estimator 기술
- `objective_metric_name = "validation:auc"` 튜닝을 하고자 하는 지표 기술
    - 이 지표의 경우는 훈련 코드에서 정의 및 기록을 해야만 합니다.
- `hyperparameter_ranges` 튜닝하고자 하는 파라미터의 범위 설정
- `max_jobs` 기술
    - 총 훈련잡의 갯수 입니다.
- `max_parallel_jobs` 기술
    - 병렬로 실행할 훈련잡의 개수 (리소스 제한에 따라서 에러가 발생할 수 있습니다. 이 경우에 줄여 주세요.)


In [10]:
objective_metric_name = "validation:auc"

pipeline_tuner = HyperparameterTuner(
    xgb_estimator, objective_metric_name, hyperparameter_ranges, 
    max_jobs=5,
    max_parallel_jobs=5,
)

## 4.6 튜닝 단계 정의 



In [11]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.model import Model
    
step_tuning = TuningStep(
    name = "Fraud-Advance-HPO",
    tuner = pipeline_tuner,
    inputs={
        "train": TrainingInput(
            s3_data= input_data,
            content_type="text/csv"
        ),
    },
)


# 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행


이제 지금까지 생성한 단계들을 하나의 파이프라인으로 조합하고 실행하도록 하겠습니다.

파이프라인은 name, parameters, steps 속성이 필수적으로 필요합니다. 
여기서 파이프라인의 이름은 (account, region) 조합에 대하여 유일(unique))해야 합니다.
우리는 또한 여기서 Experiment 설정을 추가 하여, 실험에 등록 합니다.

주의:

- 정의에 사용한 모든 파라미터가 존재해야 합니다.
- 파이프라인으로 전달된 단계(step)들은 실행순서와는 무관합니다. SageMaker Pipeline은 단계가 실행되고 완료될 수 있도록 의존관계를를 해석합니다.

## 5.1 파이프라인 정의
파이프라인 정의시에 아래 3개의 인자를 제공합니다.
- 파이프라인 이름
- 파이프라인 파라미터
- 파이프라인 실험 설정
- 스텝 정의 

In [12]:
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig

project_hpo_prefix = project_prefix + "-HPO-Step"

pipeline_name = project_prefix
pipeline = Pipeline(
    name=project_hpo_prefix,
    parameters=[
        train_instance_type,    
        train_instance_count, 
        input_data,
    ],    
    pipeline_experiment_config=PipelineExperimentConfig(
      ExecutionVariables.PIPELINE_NAME,
      ExecutionVariables.PIPELINE_EXECUTION_ID
    ),    
    steps=[step_tuning],
)

## 5.2 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [13]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'TrainInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-2-057716757052/sagemaker-webinar-pipeline-advanced/preporc/train.csv'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Fraud-Advance-HPO',
   'Type': 'Tuning',
   'Arguments': {'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
     'ResourceLimits': {'MaxNumberOfTrainingJobs': 5,
      'MaxParallelTrainingJobs': 5},
     'TrainingJobEarlyStoppingType': 'Off',
     'HyperParameterTuningJobObjective': {'Type': 'Maximize',
      'MetricName': 'validation:auc'},
     'ParameterRanges': {'ContinuousParameterRanges': [{'Name': 'eta',
        'MinValue': '0',
   

## 5.3 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [14]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()


워크플로우의 실행상황을 살펴봅니다. 

In [15]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-webinar-pipeline-advanced-hpo-step',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-webinar-pipeline-advanced-hpo-step/execution/r5mv2ctdpfqk',
 'PipelineExecutionDisplayName': 'execution-1647138924331',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 3, 13, 2, 35, 24, 221000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 3, 13, 2, 35, 24, 221000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '255e062c-0d5c-4427-914c-5c89df2183a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '255e062c-0d5c-4427-914c-5c89df2183a0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '463',
   'date': 'Sun, 13 Mar 2022 02:35:24 GMT'},
  'RetryAttempts': 0}}

## 5.4 파이프라인 실행 기다리기

In [16]:
execution.wait()

## 5.5 파이프라인 실행 단계 기록 보기

In [17]:
execution.list_steps()

[{'StepName': 'Fraud-Advance-HPO',
  'StartTime': datetime.datetime(2022, 3, 13, 2, 35, 25, 113000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 3, 13, 2, 38, 54, 276000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TuningJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:hyper-parameter-tuning-job/r5mv2ctdpfqk-fraud-a-vdz6prgnev'}}}]

# 6. HPO 잡 실행 확인 하기


## 6.1 세이지 메이커 스튜디오에서 튜닝 스텝 결과 확인 하기
실행된 파이프라인의 그래프의 HPO 스텝을 클릭하면 아래와 같은 상세 사항이 나옵니다.
- input 은 위에서 Tunner 의 정의한 내역이 보여 집니다.
- output 은 실제 HPO 잡이 수행한 후에 결과가 보입니다.

![hpo-result.png](img/hpo-result.png)

## 6.2 Python SDK 로 실행 결과 확인 하기

tunning job 중에서 가장 최근에 실행한 job name 을 가져옵니다.

In [18]:
list_tuning_job = sm_client.list_hyper_parameter_tuning_jobs(
    SortBy = 'CreationTime',
    SortOrder = 'Descending'
)
latest_tuner_job_name = list_tuning_job['HyperParameterTuningJobSummaries'][0]['HyperParameterTuningJobName']

### Best 훈련 Job 출력
- 수행된 훈련 잡 중에서 가장 성능이 좋은 훈련 잡을 기술하고, 최종 사용된 하이퍼 파리미터 값을 보여 줌

In [19]:
from pprint import pprint

# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sm_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=latest_tuner_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)
is_minimize = (
    tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["Type"] != "Maximize"
)
objective_name = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["MetricName"]

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")


5 training jobs have completed
Best model found so far:
{'CreationTime': datetime.datetime(2022, 3, 13, 2, 35, 40, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.8269000053405762},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2022, 3, 13, 2, 38, 18, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:training-job/r5mv2ctdpfqk-fraud-a-vdz6prgnev-004-08b8e61f',
 'TrainingJobName': 'r5mv2ctdpfqk-Fraud-A-VDZ6PrgneV-004-08b8e61f',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2022, 3, 13, 2, 36, 55, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.2461191309317394',
                          'eta': '0.45250843353238634',
                          'max_depth': '1',
                          'min_child_weight': '3.2805667701627588'}}


### 튜닝을 수행한 모든 훈련 잡의 결과 확인
- `FinalObjectiveValue` 의 성능 지표 순서로 보여 줌

In [20]:
import pandas as pd

tuner_df = sagemaker.HyperparameterTuningJobAnalytics(latest_tuner_job_name)

full_df = tuner_df.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 5
{'lowest': 0.7897999882698059, 'highest': 0.8269000053405762}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,1.246119,0.452508,1.0,3.280567,r5mv2ctdpfqk-Fraud-A-VDZ6PrgneV-004-08b8e61f,Completed,0.8269,2022-03-13 02:36:55+00:00,2022-03-13 02:38:18+00:00,83.0
4,1.056777,0.080591,3.0,7.424604,r5mv2ctdpfqk-Fraud-A-VDZ6PrgneV-001-09ad3a8a,Completed,0.8193,2022-03-13 02:36:56+00:00,2022-03-13 02:38:23+00:00,87.0
3,1.449919,0.673916,2.0,5.471698,r5mv2ctdpfqk-Fraud-A-VDZ6PrgneV-002-7cba551a,Completed,0.8188,2022-03-13 02:37:00+00:00,2022-03-13 02:38:23+00:00,83.0
0,0.115811,0.945107,8.0,7.274946,r5mv2ctdpfqk-Fraud-A-VDZ6PrgneV-005-e5492e67,Completed,0.7945,2022-03-13 02:36:59+00:00,2022-03-13 02:38:27+00:00,88.0
2,1.440845,0.623825,6.0,5.944257,r5mv2ctdpfqk-Fraud-A-VDZ6PrgneV-003-1ca584b6,Completed,0.7898,2022-03-13 02:37:00+00:00,2022-03-13 02:38:28+00:00,88.0
